# OSI Lat Long
### This Notebook uses the XY of the point featureclass and adds Lat Long to each related row in a related table. The FeederID (aka Circuit Number) is also determined for Electric features.
A CSV file is generated that will be used to support a seperate project.

The second half of the Notebook explores the electric consumption data with a [SpatialDataFrame](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html?highlight=spatialdataframe#spatialdataframe) using the customers table modifed in the first half.

W/WW notes: needs GloabID in customer account tables

In [30]:
#import arcpy
import numpy as np
import pandas as pd

In [16]:
gdb = "C:/Users/friendde/Documents/ArcGIS/Projects/OSILatLong/OSILatLong.gdb"
omsRegion = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eRegion"
sourceGDBList = ["C:/GISData/Data/Snapshot/mxElectric.geodatabase/","C:/GISData/Data/Snapshot/mxGas.geodatabase/",
                 "C:/arcdata/WWW_Extract.gdb/"]
esvcPntDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eServicePoint"
gsvcPntDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\gMeterSet"
rsvcPntDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\rServicePoint"
wsvcPntDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\ServicePoint"
svcPntDestList = [esvcPntDest,gsvcPntDest,rsvcPntDest,wsvcPntDest]

#svcPntDict = {esvcPntSource:esvcPntDest,gsvcPntSource:gsvcPntDest,rsvcPntSource:rsvcPntDest,wsvcPntSource:wsvcPntDest}

ecustAcctSource = r"C:\GISData\Data\Snapshot\mxElectric.geodatabase\main.eCUSTOMERACCOUNT"
gcustAcctSource = r"C:\GISData\Data\Snapshot\mxGas.geodatabase\main.gCUSTOMERACCOUNT"
rcustAcctSource = r"C:\arcdata\WWW_Extract.gdb\rCustomerAccount"
wcustAcctSource = r"C:\arcdata\WWW_Extract.gdb\wCustomerAccount"
custAcctSourceList = [ecustAcctSource,gcustAcctSource,rcustAcctSource,wcustAcctSource]

ecustAcctDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eCUSTOMERACCOUNT"
gcustAcctDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\gCUSTOMERACCOUNT"
rcustAcctDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\rCUSTOMERACCOUNT"
wcustAcctDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\wCUSTOMERACCOUNT"
custAcctDestList = [ecustAcctDest,gcustAcctDest,rcustAcctDest,wcustAcctDest]

custAcctDict = {ecustAcctSource:ecustAcctDest,gcustAcctSource:gcustAcctDest,rcustAcctSource:rcustAcctDest,wcustAcctSource:wcustAcctDest}

electFields = ["PHASEDESIGNATION","FEEDERID","TRANSFORMERBANKOBJECTID","eTransformerBank_GLOBALID","RegionName"]
svcPntDestFlds = ["OID@","GLOBALID","POINT_X","POINT_Y","PHASEDESIGNATION","FEEDERID","TRANSFORMERBANKOBJECTID","eTransformerBank_GLOBALID","RegionName"]
custAcctDestFlds = ["SERVICEPOINTOBJECTID","GLOBALID","POINT_X","POINT_Y","PHASEDESIGNATION","FeederID","Utility","TRANSFORMERBANKOBJECTID","eTransformerBank_GLOBALID","RegionName"]
custAcctOutFlds = ["OID@","SvcPntOID","INSTALL_NUM","POINT_X","POINT_Y","PHASEDESIGNATION","FeederID","TRANSFORMERBANKOBJECTID","eTransformerBank_GLOBALID","RegionName"]
custAnalysis = ["SERVICEPOINTOBJECTID","INSTALL_NUM","POINT_X","POINT_Y","AVGCONSUMPTION","MAXCONSUMPTION"]
custLocations = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\CustomerLocations"
custAcctFile = r'C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\GISAll.csv'

regions = ['SE', 'SW Main St to 34 St', 'NW Main St to 34 St', 'SW west of I-75', 'SW 34 St to I-75', 'NW west of I-75', 'NW to 39 Ave, 34 Ave to I-75', 'NW north of 39 Ave', 'NE']
expression = "updateRegion(region)"
codeblock = """
def updateRegion(name):
    return str(name)"""

#### Phase Translation between ArcFM and OSI
| Phase | ArcFM | OSI |
|-------|-------|-----|
|   A   |   4   |  1  ||
|   B   |   2   |  2  |
|   C   |   1   |  3  |
|   AB  |   6   |  12 |
|   AC  |   5   |  13 |
|   BC  |   3   |  23 |
|   ABC |   7   |  123|

In [3]:
# convert ArcFM Phase to OSI Phase
def getPhaseDesignation(phaseDesignation):
        if phaseDesignation is None:
                ph = 0
                return ph
        if phaseDesignation == 1:
                ph = 3
                return ph
        if phaseDesignation == 2:
                ph = 2
                return ph
        if phaseDesignation == 3:
                ph = 23
                return ph
        if phaseDesignation == 4:
                ph = 1
                return ph
        if phaseDesignation == 5:
                ph = 13
                return ph
        if phaseDesignation == 6:
                ph = 12
                return ph
        if phaseDesignation == 7:
                ph = 123
                return ph
def gettxOID(txOID):
        if txOID is None:
                txOID = 0
                return txOID
        else:
            return txOID
def gettxGUID(txGUID):
        if txGUID is None:
            return 0
        else:
            return txGUID

def replaceChar(strReplace):
    replaceList = [' ',',','-']
    for r in replaceList:
        if r in (strReplace):
            strReplace = strReplace.replace(r,'')

    return strReplace

def listReplace(lst, _from, _to):
   return([_to if v == _from else v for v in lst])

In [4]:
arcpy.env.workspace = gdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)

In [5]:
# delete all existing tables in workspace, new tables are created or copied in
fdsList = arcpy.ListDatasets()
for fds in fdsList:
    print(f'Deleting {fds}')
    arcpy.Delete_management(fds)
tblList = arcpy.ListTables()
for tbl in tblList:
    print(f'Deleting {tbl}')
    arcpy.Delete_management(tbl)
fcsList = arcpy.ListFeatureClasses()
for fcs in fcsList:
    if fcs == 'eRegion':
        pass
    else:
        print(f'Deleting {fcs}')
        arcpy.Delete_management(fcs)

Electric
Gas
Reclaimed
Water
eCUSTOMERACCOUNT
SAP_INSTALLATION
gCUSTOMERACCOUNT
rCUSTOMERACCOUNT
wCUSTOMERACCOUNT
Deleting SE
Deleting SWMainStto34St
Deleting NWMainStto34St
Deleting SWwestofI75
Deleting SW34SttoI75
Deleting NWwestofI75
Deleting NWto39Ave34AvetoI75
Deleting NWnorthof39Ave
Deleting NE


In [6]:
# copy in custAcctDict customer tables into workspace
# add fields not normaly in customer tables
for k,v in custAcctDict.items():
    print(f'Copying {k} to {v}')
    arcpy.Copy_management(k,v)
    if arcpy.Exists(v):
        #print(f'Copy Success {v}')
        #field_names = [f.name for f in arcpy.ListFields(v)]
        arcpy.AddField_management(v, "POINT_X", "DOUBLE")
        arcpy.AddField_management(v, "POINT_Y", "DOUBLE")
        arcpy.AddField_management(v, "PHASEDESIGNATION", "LONG")
        arcpy.AddField_management(v, "SERVICEPOINTOBJECTID", "LONG")
        arcpy.AddField_management(v, "Utility", "TEXT")
        arcpy.AddField_management(v, "FeederID", "TEXT")
        arcpy.AddField_management(v, "TRANSFORMERBANKOBJECTID", "LONG")
        arcpy.AddField_management(v, "eTransformerBank_GLOBALID", "GUID")
        arcpy.AddField_management(v, "RegionName", "TEXT")
    else:
        print(f'Copy Failed! {v}')

Copying C:\GISData\Data\Snapshot\mxElectric.geodatabase\main.eCUSTOMERACCOUNT to C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eCUSTOMERACCOUNT
Copying C:\GISData\Data\Snapshot\mxGas.geodatabase\main.gCUSTOMERACCOUNT to C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\gCUSTOMERACCOUNT
Copying C:\arcdata\WWW_Extract.gdb\rCustomerAccount to C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\rCUSTOMERACCOUNT
Copying C:\arcdata\WWW_Extract.gdb\wCustomerAccount to C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\wCUSTOMERACCOUNT


import os
for fc in svcPntDestList:
    print(os.path.split(fc)[1:])

In [7]:
#Add Responder Region Name to Service Points (electric only for now)
try:
    arcpy.AddField_management(esvcPntDest, "RegionName", "TEXT")
    arcpy.MakeFeatureLayer_management(esvcPntDest,'eSvcPnt')
    for region in regions:
        with arcpy.da.SearchCursor(omsRegion,"RegionName") as sc:
            print(f"Current Region Name {region}")
            #desc = arcpy.da.Describe(omsRegion)
            #print(f"FIDSet {desc['FIDSet']}")
            regionName = replaceChar(region)
            #print(f"New Region Name {regionName}")
            arcpy.MakeFeatureLayer_management(omsRegion,regionName,f"RegionName = '{region}'")
            arcpy.CopyFeatures_management(regionName, gdb + f"/{regionName}")
            if arcpy.Exists(gdb + f"/{regionName}"):
                #print(f"found {regionName}")
                arcpy.SelectLayerByLocation_management('eSvcPnt',"",regionName)
                #result = arcpy.GetCount_management(gdb + "/" + regionName)
                #print(result)
                result = arcpy.GetCount_management('eSvcPnt')
                print(f"{result} meters are within {region}")
                #desc = arcpy.Describe('eSvcPnt')
                #print(desc['FIDSet'])
                #print(len(desc.FIDSet.split(";")))
                if int(result.getOutput(0)) > 0:
                    print(f'Adding {region} to {int(result.getOutput(0))} Service Points')
                    arcpy.CalculateField_management('eSvcPnt',"RegionName",expression,"PYTHON",codeblock)
except:
    print(arcpy.GetMessages(2))
    

Current Region Name SE
4351 meters are within SE
Adding SE to 4351 Service Points
Current Region Name SW Main St to 34 St
4837 meters are within SW Main St to 34 St
Adding SW Main St to 34 St to 4837 Service Points
Current Region Name NW Main St to 34 St
12014 meters are within NW Main St to 34 St
Adding NW Main St to 34 St to 12014 Service Points
Current Region Name SW west of I-75
10099 meters are within SW west of I-75
Adding SW west of I-75 to 10099 Service Points
Current Region Name SW 34 St to I-75
2638 meters are within SW 34 St to I-75
Adding SW 34 St to I-75 to 2638 Service Points
Current Region Name NW west of I-75
3469 meters are within NW west of I-75
Adding NW west of I-75 to 3469 Service Points
Current Region Name NW to 39 Ave, 34 Ave to I-75
6595 meters are within NW to 39 Ave, 34 Ave to I-75
Adding NW to 39 Ave, 34 Ave to I-75 to 6595 Service Points
Current Region Name NW north of 39 Ave
7665 meters are within NW north of 39 Ave
Adding NW north of 39 Ave to 7665 Service

In [18]:
# add XY coords and some electric fields to all service point features
for fc in svcPntDestList:
    if arcpy.Exists(fc):
        print(f'Adding XY to {fc}')
        arcpy.AddXY_management(fc)
        for electField in electFields:
            if len(arcpy.ListFields(fc, electField)):
                print(f'{electField} exists in {fc}')
                pass
            else:                
                print(f'Adding {electField} to {fc}')
                if electField in ["PHASEDESIGNATION","TRANSFORMERBANKOBJECTID"]:
                    arcpy.AddField_management(fc, electField, "LONG")
                elif electField == "eTransformerBank_GLOBALID":
                    arcpy.AddField_management(fc, electField, "GUID")
                else:
                    arcpy.AddField_management(fc, electField, "TEXT")

Adding XY to C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eServicePoint
PHASEDESIGNATION exists in C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eServicePoint
FEEDERID exists in C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eServicePoint
TRANSFORMERBANKOBJECTID exists in C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eServicePoint
eTransformerBank_GLOBALID exists in C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eServicePoint
RegionName exists in C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eServicePoint
Adding XY to C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\gMeterSet
PHASEDESIGNATION exists in C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\gMeterSet
FEEDERID exists in C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\gMeterSet
TRANSFORMERBANKOBJECTID exists in C:\Users\frie

In [10]:
#check for GlobalID before migrating relationship classes
for custAcctDest in custAcctDestList:
    fldList = [fld.name.upper() for fld in arcpy.Describe(custAcctDest).fields]
    #print(fldList)
    if "GLOBALID" not in fldList:
        print(f'Adding GlobalID to {custAcctDest}')
        arcpy.AddGlobalIDs_management(custAcctDest)

In [11]:
# convert ObjectID relationship to GlobalID relationship
rcList = [c.name for c in arcpy.Describe(gdb).children if c.datatype == "RelationshipClass"]
for rc in rcList:
    desc = arcpy.Describe(rc)
    print("%-25s %s" % ("Backward Path Label:", desc.backwardPathLabel))
    print("%-25s %s" % ("Cardinality:", desc.cardinality))
    print("%-25s %s" % ("Class key:", desc.classKey))
    print("%-25s %s" % ("Destination Class Names:", desc.destinationClassNames))
    print("%-25s %s" % ("Forward Path Label:", desc.forwardPathLabel)) 
    print("%-25s %s" % ("Is Attributed:", desc.isAttributed))
    print("%-25s %s" % ("Is Composite:", desc.isComposite)) 
    print("%-25s %s" % ("Is Reflexive:", desc.isReflexive))
    print("%-25s %s" % ("Key Type:", desc.keyType))
    print("%-25s %s" % ("Notification Direction:", desc.notification))
    print("%-25s %s" % ("Origin Class Names:", desc.originClassNames))
    print("\n")

Backward Path Label:      ServicePoint
Cardinality:              OneToMany
Class key:                Undefined
Destination Class Names:  ['eCUSTOMERACCOUNT']
Forward Path Label:       Customer
Is Attributed:            False
Is Composite:             True
Is Reflexive:             False
Key Type:                 Single
Notification Direction:   Forward
Origin Class Names:       ['eServicePoint']


Backward Path Label:      SAPInstallation
Cardinality:              OneToOne
Class key:                Undefined
Destination Class Names:  ['SAP_INSTALLATION']
Forward Path Label:       eCustomerAccount
Is Attributed:            False
Is Composite:             True
Is Reflexive:             False
Key Type:                 Single
Notification Direction:   Both
Origin Class Names:       ['eCUSTOMERACCOUNT']


Backward Path Label:      MeterSet
Cardinality:              OneToMany
Class key:                Undefined
Destination Class Names:  ['gCUSTOMERACCOUNT']
Forward Path Label:       Customer

In [12]:
for rc in rcList:
    print(f'Migrating {rc}')
    arcpy.MigrateRelationshipClass_management(rc)

Migrating ServicePt_CustomerAcct
Migrating eCustomeraccount_SAPInstallation
Migrating gMeterSet_gCustomerAccount
Migrating RSERVICEPOINT_CUSTACCT
Migrating WSERVICEPOINT_CUSTACCT


In [49]:
edit = arcpy.da.Editor(gdb)
edit.startEditing(False, False)
edit.startOperation()

field_names = [f.name for f in arcpy.ListFields(gsvcPntDest)]
print(field_names)

In [50]:
for rc in rcList:
    desc = arcpy.Describe(rc)
    if desc.originClassNames[0] == "SAP_INSTALLATION":
        pass
    elif desc.destinationClassNames[0] == "SAP_INSTALLATION":
        pass
    else:
        custAcctFlds = []
        custAcctFlds = listReplace(custAcctDestFlds,"GLOBALID",f"{desc.originClassNames[0]}_GLOBALID")
        with arcpy.da.SearchCursor(desc.originClassNames[0],svcPntDestFlds) as svcpnts:
#["OID@","GLOBALID","POINT_X","POINT_Y","PHASEDESIGNATION","FEEDERID","TRANSFORMERBANKOBJECTID","eTransformerBank_GLOBALID","RegionName"]
            print(f"Searching {desc.originClassNames[0]} and updating {desc.destinationClassNames[0]}")
            for svcpnt in svcpnts:
                whereClause = f"{desc.originClassNames[0]}_GLOBALID = '{svcpnt[1]}'"
                with arcpy.da.UpdateCursor(desc.destinationClassNames[0],custAcctDestFlds,whereClause) as uc:
#["SERVICEPOINTOBJECTID","GLOBALID","POINT_X","POINT_Y","PHASEDESIGNATION","FeederID","Utility","TRANSFORMERBANKOBJECTID","eTransformerBank_GLOBALID","RegionName"]
                    #print(f"Updating: {desc.destinationClassNames[0]}")
                    for row in uc:
                        row[0] = svcpnt[0] #SncPntOID
                        #row[1] = svcpnt[1] #GlobalID
                        row[2] = svcpnt[2] #Point_X
                        row[3] = svcpnt[3] #Point_Y
                        row[4] = getPhaseDesignation(svcpnt[4]) #Phase
                        row[5] = svcpnt[5] #FeederID
                        row[6] = desc.destinationClassNames[0][:1] #Utility from CustomerAccount table prefix letter
                        #row[7] = gettxOID(svcpnt[7])
                        row[8] = gettxGUID(svcpnt[7]) #eTx_GlobalID
                        row[9] = svcpnt[8] #RegionName
                        uc.updateRow(row)

Searching eServicePoint and updating eCUSTOMERACCOUNT
Searching gMeterSet and updating gCUSTOMERACCOUNT
Searching rServicePoint and updating rCUSTOMERACCOUNT
Searching ServicePoint and updating wCUSTOMERACCOUNT


In [51]:
edit.stopOperation()
edit.stopEditing(True)

In [23]:
#arcpy.Delete_management(custLocations)
arcpy.CreateTable_management(gdb,"CustomerLocations",ecustAcctDest)

<Result 'C:\\Users\\friendde\\Documents\\ArcGIS\\Projects\\OSILatLong\\OSILatLong.gdb\\CustomerLocations'>

In [52]:
arcpy.TruncateTable_management(custLocations)
arcpy.Append_management(custAcctDestList,custLocations,"NO_TEST")

<Result 'C:\\Users\\friendde\\Documents\\ArcGIS\\Projects\\OSILatLong\\OSILatLong.gdb\\CustomerLocations'>

In [53]:
expression = "setFeeder(!FeederID!)"
codeblock = """
def setFeeder(feeder):
    if feeder is None:
        return 0
    else:
        return feeder"""
arcpy.CalculateField_management(custLocations,"FeederId",expression,"PYTHON3",codeblock)

<Result 'C:\\Users\\friendde\\Documents\\ArcGIS\\Projects\\OSILatLong\\OSILatLong.gdb\\CustomerLocations'>

In [77]:
expression = "setUnknown(!RegionName!)"
codeblock = """
def setUnknown(r):
    if r is None:
        return "Not Set"
    elif r == 0:
        return "Not Set"
    else:
        return r"""
arcpy.CalculateField_management(custLocations,"RegionName",expression,"PYTHON3",codeblock)

<Result 'C:\\Users\\friendde\\Documents\\ArcGIS\\Projects\\OSILatLong\\OSILatLong.gdb\\CustomerLocations'>

In [70]:
expression = "setPhase(!PhaseDesignation!)"
codeblock = """
def setPhase(phase):
    if phase is None:
        return 0
    else:
        return phase"""
arcpy.CalculateField_management(custLocations,"PhaseDesignation",expression,"PYTHON3",codeblock)

<Result 'C:\\Users\\friendde\\Documents\\ArcGIS\\Projects\\OSILatLong\\OSILatLong.gdb\\CustomerLocations'>

### Use numpy and pandas to export to CSV

Use arcpy [```TableToNumPyArray()```](http://pro.arcgis.com/en/pro-app/arcpy/data-access/tabletonumpyarray.htm)
See also [Working with numpy in ArcGIS](http://pro.arcgis.com/en/pro-app/arcpy/get-started/working-with-numpy-in-arcgis.htm)

nparr = arcpy.da.TableToNumPyArray(custLocations,custAcctOutFlds,skip_nulls=True)
#nparr = arcpy.da.TableToNumPyArray(custAcctDest,custAcctOutFlds,null_value=-9999)
pdarr = pd.DataFrame(nparr)
pdarr.to_csv(custAcctFile,header=False, index=False)

TODO - write file to \\gruadmin.gru.com\fs\Groups\OMS Replacement Project\Documents for OSII\Customer and Premise Files

Ready new numpy array for consumption analysis

remove empty line at end of CSV

In [ ]:
#nparr = arcpy.da.TableToNumPyArray(custLocations,["ServicePointObjectID","POINT_X","POINT_Y","AVGCONSUMPTION","MAXCONSUMPTION","Utility"],skip_nulls=True)

DataFrame.drop_duplicates(subset=None, keep='first', inplace=False

In [78]:
nparr = arcpy.da.TableToNumPyArray(custLocations,["ObjectID","INSTALL_NUM","POINT_X","POINT_Y","PHASEDESIGNATION","FeederID","Utility","eTransformerBank_GLOBALID","RegionName"])

In [79]:
df = pd.DataFrame(nparr)

In [80]:
df.head()

,ObjectID,INSTALL_NUM,POINT_X,POINT_Y,PHASEDESIGNATION,FeederID,Utility,eTransformerBank_GLOBALID,RegionName
0,1,5000226717,-82.255216,29.660429,2,207,e,{7CCCBA17-54EF-42AA-9A87-02159ACDA324},NE
1,2,5000226733,-82.256019,29.659257,2,207,e,{ADB0EEA9-99B5-476D-9F44-39730CB65611},NE
2,3,5000010807,-82.253436,29.659716,2,207,e,{DFDB886D-DEB4-43BC-8C61-7DBD12C22C2F},NE
3,4,5000295952,-82.243694,29.702255,2,0,e,{D90B530F-6540-4FDB-8B46-A8D7FA6797A9},NE
4,5,5000286178,-82.253930,29.676644,2,434,e,{BBE0CC70-FE66-44B2-802F-EBE0A2E7B444},NE


In [81]:
df.tail()

,ObjectID,INSTALL_NUM,POINT_X,POINT_Y,PHASEDESIGNATION,FeederID,Utility,eTransformerBank_GLOBALID,RegionName
203768,203854,5000323217,-82.364258,29.646300,0,0,w,None,0
203769,203855,5000345778,-82.363735,29.649041,0,0,w,None,0
203770,203856,5000205415,-82.364758,29.656020,0,0,w,None,0
203771,203857,5000371121,-82.366248,29.654611,0,0,w,None,0
203772,203858,5000033678,-82.363783,29.653377,0,0,w,None,0


Show only rows with a series filter

In [82]:
df.loc[(df["Utility"]=="g"), ["ObjectID","INSTALL_NUM","POINT_X","POINT_Y","PHASEDESIGNATION","FeederID","Utility","eTransformerBank_GLOBALID","RegionName"]]

,ObjectID,INSTALL_NUM,POINT_X,POINT_Y,PHASEDESIGNATION,FeederID,Utility,eTransformerBank_GLOBALID,RegionName
96300,96354,5000231889,-82.331234,29.683879,0,0,g,None,0
96301,96355,5000232034,-82.331028,29.683705,0,0,g,None,0
96302,96356,5000070502,-82.329524,29.687743,0,0,g,None,0
96303,96357,5000070055,-82.330427,29.687203,0,0,g,None,0
96304,96358,5000070024,-82.330208,29.687193,0,0,g,None,0
96305,96359,5000069986,-82.329839,29.687183,0,0,g,None,0
96306,96360,5000069771,-82.328902,29.686731,0,0,g,None,0
96307,96361,5000069739,-82.329150,29.686765,0,0,g,None,0
96308,96362,5000069697,-82.329488,29.686765,0,0,g,None,0
96309,96363,5000069596,-82.329788,29.686709,0,0,g,None,0


In [83]:
df.ObjectID

0              1
1              2
2              3
3              4
4              5
5              6
6              7
7              8
8              9
9             10
10            11
11            12
12            13
13            14
14            15
15            16
16            17
17            18
18            19
19            20
20            21
21            22
22            23
23            24
24            25
25            26
26            27
27            28
28            29
29            30
           ...  
203743    203829
203744    203830
203745    203831
203746    203832
203747    203833
203748    203834
203749    203835
203750    203836
203751    203837
203752    203838
203753    203839
203754    203840
203755    203841
203756    203842
203757    203843
203758    203844
203759    203845
203760    203846
203761    203847
203762    203848
203763    203849
203764    203850
203765    203851
203766    203852
203767    203853
203768    203854
203769    203855
203770    2038

In [84]:
df = df.astype({'ObjectID':str}, copy=False)

In [ ]:
df.ObjectID

In [85]:
df.ObjectID = np.where(df.Utility != 'e', df.Utility + df.ObjectID,df.ObjectID)

In [86]:
df.head()

,ObjectID,INSTALL_NUM,POINT_X,POINT_Y,PHASEDESIGNATION,FeederID,Utility,eTransformerBank_GLOBALID,RegionName
0,1,5000226717,-82.255216,29.660429,2,207,e,{7CCCBA17-54EF-42AA-9A87-02159ACDA324},NE
1,2,5000226733,-82.256019,29.659257,2,207,e,{ADB0EEA9-99B5-476D-9F44-39730CB65611},NE
2,3,5000010807,-82.253436,29.659716,2,207,e,{DFDB886D-DEB4-43BC-8C61-7DBD12C22C2F},NE
3,4,5000295952,-82.243694,29.702255,2,0,e,{D90B530F-6540-4FDB-8B46-A8D7FA6797A9},NE
4,5,5000286178,-82.253930,29.676644,2,434,e,{BBE0CC70-FE66-44B2-802F-EBE0A2E7B444},NE


In [87]:
df.tail()

,ObjectID,INSTALL_NUM,POINT_X,POINT_Y,PHASEDESIGNATION,FeederID,Utility,eTransformerBank_GLOBALID,RegionName
203768,w203854,5000323217,-82.364258,29.646300,0,0,w,None,0
203769,w203855,5000345778,-82.363735,29.649041,0,0,w,None,0
203770,w203856,5000205415,-82.364758,29.656020,0,0,w,None,0
203771,w203857,5000371121,-82.366248,29.654611,0,0,w,None,0
203772,w203858,5000033678,-82.363783,29.653377,0,0,w,None,0


In [88]:
df.loc[(df.Utility == 'w'), ["ObjectID","INSTALL_NUM","POINT_X","POINT_Y","PHASEDESIGNATION","FeederID","Utility","eTransformerBank_GLOBALID","RegionName"]]

,ObjectID,INSTALL_NUM,POINT_X,POINT_Y,PHASEDESIGNATION,FeederID,Utility,eTransformerBank_GLOBALID,RegionName
133186,w133247,5000326532,-82.372671,29.692579,0,0,w,None,0
133187,w133248,5000326490,-82.372725,29.692580,0,0,w,None,0
133188,w133249,5000008681,-82.373142,29.692581,0,0,w,None,0
133189,w133250,5000293743,-82.373197,29.692582,0,0,w,None,0
133190,w133251,5000020548,-82.373414,29.692582,0,0,w,None,0
133191,w133252,5000026101,-82.373774,29.692767,0,0,w,None,0
133192,w133253,5000326329,-82.373794,29.692883,0,0,w,None,0
133193,w133254,5000020484,-82.373892,29.693347,0,0,w,None,0
133194,w133255,5000326285,-82.373947,29.693582,0,0,w,None,0
133195,w133256,5000293711,-82.373943,29.693559,0,0,w,None,0


In [ ]:
#df = df.astype({'INSTALL_NUM':str}, copy=True)
df2 = df.astype(str, copy=True)
df2.INSTALL_NUM

#df.groupby(df.columns.tolist(),as_index=False).size()
grouped = df.groupby('Utility')
for name,group in grouped:
    print(name)
    #print(group)
    #df.groupby('INSTALL_NUM').filter(lambda x: len(x) > 1).drop_duplicates(subset='INSTALL_NUM')
    #df2.groupby(['INSTALL_NUM', 'Utility']).filter(lambda x: x if len(x) > 1 and 'Utility' == name else None)
    #print(df.groupby('INSTALL_NUM').filter(lambda x: x if len(x) > 1 and 'Utility' == name else None))
    #print(df.groupby('INSTALL_NUM').filter(lambda x: len(x) > 1 and 'Utility' == name))
    print(df.groupby('INSTALL_NUM').filter(lambda x: len(x) > 1)).drop_duplicates(subset='INSTALL_NUM')

In [89]:
df.drop_duplicates(subset='INSTALL_NUM', keep='first', inplace=True)

In [90]:
df.groupby('INSTALL_NUM').filter(lambda x: len(x) > 1)

,ObjectID,INSTALL_NUM,POINT_X,POINT_Y,PHASEDESIGNATION,FeederID,Utility,eTransformerBank_GLOBALID,RegionName


In [91]:
df.groupby(['INSTALL_NUM']).size().reset_index(name='count')

,INSTALL_NUM,count
0,4199 NW 86 TER,1
1,5000001182,1
2,5000001183,1
3,5000001185,1
4,5000001187,1
5,5000001189,1
6,5000001193,1
7,5000001194,1
8,5000001197,1
9,5000001198,1


In [92]:
df.drop(['Utility'], axis=1, inplace=True)

In [93]:
df.tail()

,ObjectID,INSTALL_NUM,POINT_X,POINT_Y,PHASEDESIGNATION,FeederID,eTransformerBank_GLOBALID,RegionName
203768,w203854,5000323217,-82.364258,29.646300,0,0,None,0
203769,w203855,5000345778,-82.363735,29.649041,0,0,None,0
203770,w203856,5000205415,-82.364758,29.656020,0,0,None,0
203771,w203857,5000371121,-82.366248,29.654611,0,0,None,0
203772,w203858,5000033678,-82.363783,29.653377,0,0,None,0


In [94]:
df.to_csv(custAcctFile,header=False, index=True)

[```gis.features.SpatialDataFrame()```](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html?highlight=spatialdataframe#arcgis.features.SpatialDataFrame.from_xy)

In [ ]:
from arcgis.features import SpatialDataFrame
from arcgis.gis import GIS
from getpass import getpass
from IPython.display import display

In [ ]:
sdf = SpatialDataFrame.from_xy(df,"POINT_X","POINT_Y")
gis = GIS(arcpy.GetActivePortalURL(), username=input("Enter User Name "), password=(getpass()))
#gis = GIS()
#portalDesc = arcpy.GetPortalDescription()
# search and list all items owned by connected user
#query=f'owner:{portalDesc["user"]["username"]} AND title:CW BaseMap'
#itemType="Feature Layer"
#sortField="title"
#sortOrder="asc"
# default max__items is 10
#maxItems=100
#m = gis.content.search(query,itemType,sortField,sortOrder,maxItems)

In [ ]:
consumptionLyr = gis.content.import_data(sdf)

In [ ]:
m = gis.map('Gainesville,FL')

In [ ]:
m.add_layer(sdf,options={"renderer":"ClassedSizeRenderer","field_name":"MAXCONSUMPTION"})

In [ ]:
m.add_layer(consumptionLyr,options={"renderer":"ClassedSizeRenderer","field_name":"MAXCONSUMPTION"})

In [ ]:
m